## Demo of DuckDB

### Demo Goals:
##### - Avoid using pandas dataframes to load and query b/c won't work when data > memory.
##### - Take advantage of DuckDB's super fast data loading.
##### - Directly load external data into a DuckDB database, then query.
##### - Using pandas for small query results is Ok.

<span style="color:LIGHTGREEN">Import the DuckDB package</span>

In [68]:
import duckdb

<span style="color:LIGHTGREEN">Very basic test...</span>

In [69]:
duckdb.sql("SELECT 42").show()

┌───────┐
│  42   │
│ int32 │
├───────┤
│    42 │
└───────┘



<span style="color:LIGHTBLUE">Because we want to keep the data we import for future use, we will create a permanent database file for it.</span>

<span style="color:LIGHTGREEN">Create a database connection...</span>

In [70]:
import duckdb

conn = duckdb.connect('taxidbdemo.db')

<span style="color:LIGHTGREEN">Load data into a DuckDB database table.</span>

In [71]:
conn.execute('''CREATE OR REPLACE TABLE taxi_data AS 
                SELECT * FROM read_parquet('yellow_tripdata_2024-01.parquet');''')

We can do some things directly on our new table.

In [72]:
conn.table('taxi_data').show(max_rows = 4)

┌──────────┬──────────────────────┬──────────────────────┬───┬──────────────┬──────────────────────┬─────────────┐
│ VendorID │ tpep_pickup_datetime │ tpep_dropoff_datet…  │ … │ total_amount │ congestion_surcharge │ Airport_fee │
│  int32   │      timestamp       │      timestamp       │   │    double    │        double        │   double    │
├──────────┼──────────────────────┼──────────────────────┼───┼──────────────┼──────────────────────┼─────────────┤
│        2 │ 2024-01-01 00:57:55  │ 2024-01-01 01:17:43  │ … │         22.7 │                  2.5 │         0.0 │
│        1 │ 2024-01-01 00:03:00  │ 2024-01-01 00:09:36  │ … │        18.75 │                  2.5 │         0.0 │
│        · │          ·           │          ·           │ · │          ·   │                   ·  │          ·  │
│        · │          ·           │          ·           │ · │          ·   │                   ·  │          ·  │
│        · │          ·           │          ·           │ · │          ·   │   

We can run any sql query we want with our new table.

fetch_df() returns a pandas dataframe but make sure it can fit into memory.

In [73]:
conn.execute("SELECT * FROM taxi_data limit 4").fetchdf()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1,1.72,1,N,186,79,2,17.7,1.0,0.5,0.00,0.0,1.0,22.70,2.5,0.0
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1,1.80,1,N,140,236,1,10.0,3.5,0.5,3.75,0.0,1.0,18.75,2.5,0.0
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1,4.70,1,N,236,79,1,23.3,3.5,0.5,3.00,0.0,1.0,31.30,2.5,0.0
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1,1.40,1,N,79,211,1,10.0,3.5,0.5,2.00,0.0,1.0,17.00,2.5,0.0


fetchall() returns all rows as list of tuples.

In [74]:
conn.execute('''SELECT VendorID, ROUND(SUM(total_amount), 2) as total 
                FROM taxi_data GROUP BY VendorID''').fetchall()

[(1, 18841261.98), (2, 60602721.27), (6, 12401.03)]

fetchdf() returns a pandas dataframe.

In [75]:
conn.execute('''SELECT VendorID, ROUND(SUM(total_amount), 2) as total 
                FROM taxi_data GROUP BY VendorID''').fetchdf()

,VendorID,total
0,1,18841261.98
1,2,60602721.27
2,6,12401.03


Extract a subset of data to use in pandas.

In [76]:
taxidf1 = conn.execute('''SELECT * FROM taxi_data WHERE VendorID = 1''').fetchdf()

In [77]:
type(taxidf1)

pandas.core.frame.DataFrame

Use the pandas dataframe.

In [78]:
taxidf1[['VendorID','trip_distance']]

,VendorID,trip_distance
0,1,1.8
1,1,4.7
2,1,1.4
3,1,0.8
4,1,4.7
...,...,...
729727,1,0.7
729728,1,1.2
729729,1,3.9
729730,1,4.0


#### DuckDB Relation Object

We can create a Relation Object in DuckDB which is a bit like a DuckDB dataframe but is not limited o your computer's memory.

Below we are using the default dattabase connection which is in memory.

In [79]:
taxi_rel = duckdb.read_parquet("yellow_tripdata_2024-01.parquet")

In [80]:
type(taxi_rel)

duckdb.duckdb.DuckDBPyRelation

### Useful Reference to functions...
https://duckdb.org/docs/api/python/reference/

In [81]:
taxi_rel.show(max_rows = 2)

┌──────────┬──────────────────────┬──────────────────────┬───┬──────────────┬──────────────────────┬─────────────┐
│ VendorID │ tpep_pickup_datetime │ tpep_dropoff_datet…  │ … │ total_amount │ congestion_surcharge │ Airport_fee │
│  int32   │      timestamp       │      timestamp       │   │    double    │        double        │   double    │
├──────────┼──────────────────────┼──────────────────────┼───┼──────────────┼──────────────────────┼─────────────┤
│        2 │ 2024-01-01 00:57:55  │ 2024-01-01 01:17:43  │ … │         22.7 │                  2.5 │         0.0 │
│        · │          ·           │          ·           │ · │           ·  │                   ·  │          ·  │
│        · │          ·           │          ·           │ · │           ·  │                   ·  │          ·  │
│        · │          ·           │          ·           │ · │           ·  │                   ·  │          ·  │
│        2 │ 2024-01-01 01:22:05  │ 2024-01-01 01:33:29  │ … │         17.8 │   

In [82]:
duckdb.sql("SELECT * FROM taxi_rel").show(max_rows = 5)

┌──────────┬──────────────────────┬──────────────────────┬───┬──────────────┬──────────────────────┬─────────────┐
│ VendorID │ tpep_pickup_datetime │ tpep_dropoff_datet…  │ … │ total_amount │ congestion_surcharge │ Airport_fee │
│  int32   │      timestamp       │      timestamp       │   │    double    │        double        │   double    │
├──────────┼──────────────────────┼──────────────────────┼───┼──────────────┼──────────────────────┼─────────────┤
│        2 │ 2024-01-01 00:57:55  │ 2024-01-01 01:17:43  │ … │         22.7 │                  2.5 │         0.0 │
│        1 │ 2024-01-01 00:03:00  │ 2024-01-01 00:09:36  │ … │        18.75 │                  2.5 │         0.0 │
│        1 │ 2024-01-01 00:17:06  │ 2024-01-01 00:35:01  │ … │         31.3 │                  2.5 │         0.0 │
│        · │          ·           │          ·           │ · │           ·  │                   ·  │          ·  │
│        · │          ·           │          ·           │ · │           ·  │   

Get some summary stats on our relation.

In [83]:
taxi_rel.describe()

┌─────────┬─────────────────────┬──────────────────────┬───┬──────────────────────┬────────────────────┐
│  aggr   │      VendorID       │ tpep_pickup_datetime │ … │ congestion_surcharge │    Airport_fee     │
│ varchar │       double        │       varchar        │   │        double        │       double       │
├─────────┼─────────────────────┼──────────────────────┼───┼──────────────────────┼────────────────────┤
│ count   │           2964624.0 │ 2964624              │ … │            2824462.0 │          2824462.0 │
│ mean    │  1.7542042431013174 │ NULL                 │ … │   2.2561220508542865 │  0.141161130863152 │
│ stddev  │ 0.43259020170360285 │ NULL                 │ … │   0.8232746699398348 │ 0.4876238872392802 │
│ min     │                 1.0 │ 2002-12-31 22:59:39  │ … │                 -2.5 │              -1.75 │
│ max     │                 6.0 │ 2024-02-01 00:01:15  │ … │                  2.5 │               1.75 │
│ median  │                 2.0 │ NULL                 

In [84]:
conn.close()